<a href="https://colab.research.google.com/github/Ezhil531/image-classification/blob/main/ml_minor_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet

In [12]:
!pip install bing-image-downloader

In [16]:
!mkdir images

mkdir: cannot create directory ‘images’: File exists


In [14]:
from bing_image_downloader import downloader
downloader.download("Cricket Bat",limit=30,output_dir='images',adult_filter_off=True)

[%] Downloading Images to /content/images/Cricket Bat


[!!]Indexing page: 1

[%] Indexed 30 Images on Page 1.


[%] Downloading Image #1 from https://n3.sdlcdn.com/imgs/g/r/s/spartan-blue-popular-willow-cricket-SDL568742857-1-ccf26.jpeg
[%] File Downloaded !

[%] Downloading Image #2 from https://n2.sdlcdn.com/imgs/i/x/b/Popular-willow-cricket-bat-Market-SDL943524258-1-2348e.jpeg
[%] File Downloaded !

[%] Downloading Image #3 from https://n2.sdlcdn.com/imgs/h/8/3/Strauss-English-Willow-Cricket-Bat-SDL263160843-2-e975d.jpeg
[%] File Downloaded !

[%] Downloading Image #4 from https://n4.sdlcdn.com/imgs/i/9/l/BMD-POPLER-WELLOW-CRICKET-BAT-SDL196099538-1-cf279.jpeg
[%] File Downloaded !

[%] Downloading Image #5 from https://www.talentcricket.co.uk/images/custom/newberyinvictusbat192__size-800-0.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://n4.sdlcdn.com/imgs/j/b/n/cricket-bat-SDL597858602-1-ce327.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://n3.sdl

In [15]:
from bing_image_downloader import downloader
downloader.download("Playing Ball",limit=30,output_dir='images',adult_filter_off=True)

[%] Downloading Images to /content/images/Playing Ball


[!!]Indexing page: 1

[%] Indexed 30 Images on Page 1.


[%] Downloading Image #1 from https://i.pinimg.com/originals/c2/62/8e/c2628e0a0cd5d87e91bf56139ac506e4.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://i.pinimg.com/originals/8f/d6/20/8fd62045f5e27fb62ee5eb13bc5fe17e.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://api.awishcometrue.com/Data/Media/c32bee95-6373-41fd-aa30-af59686b098213980_B.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://i.pinimg.com/736x/a6/b1/02/a6b102b70b556f62c861bab922bb93f1--jazz-costumes-costume-ideas.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://i.pinimg.com/474x/5c/3d/d3/5c3dd3470bb9e23b17c5727b39996d31--frisbee-my-girl.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://onceuponadiamondmovie.com/wp-content/uploads/2020/01/Dean-Corson-batting-6-e1579482293374.jpg
[%] File Downloaded !

[%] Downloading Image #7 from htt

In [ ]:
#Packages
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn import svm
import pickle
from pyngrok import ngrok
from PIL import Image

In [18]:
#Pre-processing
target = []
images = []
flat_data = []

datadir = '/content/images'
categories = ['Cricket Bat','Playing Ball']

for i in categories:
  class_num = categories.index(i)
  path = os.path.join(datadir,i)
  for img in os.listdir(path):
    if img.endswith('.jpg') or img.endswith('.jpeg') or img.endswith('.png'):
      img_arr = imread(os.path.join(path,img))
      img_resized = resize(img_arr,(150,150,3))
      flat_data.append(img_resized.flatten())
      images.append(img_resized)
      target.append(class_num)

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

In [ ]:
unique, count = np.unique(target,return_counts=True)
plt.grid()
plt.bar(categories,count)

In [ ]:
#Data Spliting
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.2,random_state=120)

In [ ]:
param_grid = [ {'C':[1,10,100,1000],'kernel':['linear']},{'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']} ]
svc = svm.SVC(probability = True)
clf = GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
 accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
pickle.dump(clf,open('ImageRecog.p','wb'))

In [ ]:
model = pickle.load(open('ImageRecog.p','rb'))

In [ ]:
flat_data=[]
url=input("Enter your URL:")
img = imread(url)
img_resized=resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data=np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out = model.predict(flat_data)
print(y_out[0])
print(type(y_out[0]))
y_out=categories[y_out[0]]
print(f'PREDICTED OUTPUT IS:{y_out}')

In [ ]:
%%writefile ImageClassifier.py
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from PIL import Image
import pickle

st.title('Bat and Ball detector using M.L.')
st.text('Upload to predict.....')

model=pickle.load(open('ImageRecog.p','rb'))
uploaded_file=st.file_uploader("Upload an Image:",type="jpg")
if uploaded_file is not None:
  img=Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')
  if st.button('PREDICT'):
    categories = ['Cricket Bat','Playing Ball']
    st.write('Result: ')
    flat_data = []
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    print(img.shape)
    plt.imshow(img_resized)
    y_out = model.predict(flat_data)
    y_out = [y_out[0]]
    st.title(f'Predicted Output Is: {categories[y_out[0]]} ')
    percent = model.predict_proba(flat_data)
    for index, item in enumerate(categories):
      st.write(f'{ item } : {percent[0][index]*100}%')


In [ ]:
!nohup streamlit run ImageClassifier.py &

url = ngrok.connect(port='8501')
print(url)